In [ ]:
import sys
from pathlib import Path
import os
import json

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

In [ ]:
tracklets_to_process_override_batch1 = [i for i in range(0, 711)] # COLIN
tracklets_to_process_override_batch2 = [i for i in range(712, 1426)] # JARED Second half of the tracklets (605 items)
#merge_cache = [i for i in range(0, 1426)]

pipeline = CentralPipeline(
  num_tracklets=1211,
  #num_images_per_tracklet=50,
  #tracklets_to_process_override=tracklets_to_process_override_batch1,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  use_cache=True, # Set to false to rebuild the cache
  suppress_logging=False,
  use_image_enhancement=False,
  
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=2,                   # CRITICAL optimisation param. Adjust accordingly. STR: 1 | Everything else: 2
  tracklet_batch_size=32,          # CRITICAL optimisation param. Adjust accordingly. Always 32
  image_batch_size=100,            # CRITICAL optimisation param. Adjust accordingly. STR: 50 | Everything else: 100-200
  num_threads_multiplier=4         # CRITICAL optimisation param. Adjust accordingly.
  )                                # -> Interpretation: num_threads = num_workers * num_threads_multiplier

In [ ]:
#pipeline.aggregate_legibility_results_data()
#pipeline.aggregate_pose()
#pipeline.aggregate_str_results()

In [ ]:
pipeline.run_soccernet(
  run_soccer_ball_filter=False,
  generate_features=True,
  run_filter=False,
  run_legible=True,
  run_legible_eval=True,
  run_pose=True,
  run_crops=True,
  run_str=True,
  run_combine=False,
  run_eval=False,
  use_clip4str=True,
  pyscript=False,
  )

## Benchmark (current pipeline metrics)
### Results
- 2025-03-29 14:44:05 [INFO] Correct 1051 out of 1210. Accuracy 86.85950413223141%.
- 2025-03-29 14:44:05 [INFO] TP=766, TN=285, FP=70, FN=89
- 2025-03-29 14:44:05 [INFO] Precision=0.916267942583732, Recall=0.895906432748538
- 2025-03-29 14:44:05 [INFO] F1=0.9059727971614429
- 2025-03-29 14:44:05 [INFO] Saved failed legibility cases to:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\test\common_data\failed_legibility_cases.json

### Analysis
legible_but_marked_illegible: 89 cases
109, 88, 34, 151, 19, 68, 67, 128, 57, 178, 189, 196, 214, 242, 258, 271, 304, 327, 320, 352, 375, 378, 414, 416, 439, 425, 451, 453, 454, 466, 499, 511, 513, 526, 528, 567, 576, 594, 619, 623, 646, 687, 682, 709, 711, 717, 739, 758, 767, 795, 788, 840, 843, 832, 841, 852, 847, 851, 856, 850, 868, 866, 871, 901, 902, 904, 900, 907, 928, 939, 944, 961, 984, 983, 1005, 1039, 1064, 1063, 1077, 1091, 1101, 1137, 1152, 1153, 1160, 1167, 1157, 1165, 1206

illegible_but_marked_legible: 70 cases
168, 167, 162, 97, 39, 87, 89, 134, 61, 172, 164, 199, 215, 260, 267, 269, 281, 276, 288, 295, 317, 325, 340, 345, 374, 394, 388, 386, 391, 406, 408, 434, 437, 459, 471, 514, 532, 558, 683, 748, 770, 790, 802, 827, 861, 869, 879, 897, 895, 909, 964, 981, 999, 1023, 1031, 1025, 1033, 1030, 1055, 1088, 1087, 1169, 1175, 1174, 1173, 1183, 1170, 1179, 1184, 1185

- Total misclassifications: 159
- False negatives (legible marked as illegible): 89
- False positives (illegible marked as legible): 70
- Percentage of false negatives: 7.36%
- Percentage of false positives: 5.79%
- Combined percentage of errors as a percentage of total cases: 13.14%

In [ ]:
# import json
# failed_cases = f"{DataPaths.COMMON_PROCESSED_OUTPUT_DATA_CHALLENGE.value}/failed_legibility_cases.json"

# # Load this JSON file
# with open(failed_cases, 'r') as f:
#     failed_cases_dict = json.load(f)
#     # Print the every key and its associated value (value will be a list so need to make that printable with ",".join):
#     for key, value in failed_cases_dict.items():
#         print(f"{key}: {len(value)} cases")
#         if len(value) > 0:
#             print(f"{', '.join(value)}")
#         print()
    
#     # Summary statistics
#     legible_but_marked_illegible = len(failed_cases_dict.get("legible_but_marked_illegible", []))
#     illegible_but_marked_legible = len(failed_cases_dict.get("illegible_but_marked_legible", []))
#     total_errors = legible_but_marked_illegible + illegible_but_marked_legible
    
#     print(f"Total misclassifications: {total_errors}")
#     print(f"False negatives (legible marked as illegible): {legible_but_marked_illegible}")
#     print(f"False positives (illegible marked as legible): {illegible_but_marked_legible}")
#     print(f"Percentage of false negatives: {legible_but_marked_illegible / 1210 * 100:.2f}%")
#     print(f"Percentage of false positives: {illegible_but_marked_legible / 1210 * 100:.2f}%")
#     print(f"Combined percentage of errors as a percentage of total cases: {total_errors / 1210 * 100:.2f}%")

In [ ]:
# import timm
# import torch
# from torch import nn

# model = timm.create_model("timm/vit_base_patch16_224.orig_in21k_ft_in1k", pretrained=False)
# model.head = nn.Linear(model.head.in_features, 10)
# model.load_state_dict(
#     torch.hub.load_state_dict_from_url(
#         "https://huggingface.co/edadaltocg/vit_base_patch16_224_in21k_ft_svhn/resolve/main/pytorch_model.bin",
#         map_location="cpu",
#         file_name="vit_base_patch16_224_in21k_ft_svhn.pth",
#     )
# )